#Challenge Técnico - Data Scientist - YOM  



Alejandro Espinoza

aa.espinoza.perez@gmail.com

http://linkedin.com/in/aaep/

Lunes 23 de septiembre, 2024

## Contexto e Introducción

A continuación se presenta el notebook de python para el desarrollo del Challenge Técnico para el proceso de postulación al cargo de Data Scientist en YOM.

El desafío consta del desarrollo de un sistema de recomendación para la empresa Mayorista11 con el fin de optimizar las recomendaciones para comercios locales. Para esto contamos con 3 archivos de datos en formato CSV, los cuales se encuentran en la ruta '/content/data' y contienen:
  1. Historial de transacciones de cada comercio
  2. Información de los comercios
  3. Información de productos

El trabajo desarrollado a continuación se abordara en 3 secciones, empezando por la exploración de la data con el fin de entender su contenido, caracteristicas estadisticas, calidad del dato y procesamiento de nulos en caso de ser necesario.

En la segunda sección se aborda la division de los comercios por comuna, con el objetivo de poder asignar caracteristicas difinitorias a cada comuna y poder establecer similitudes entre productos y comercios.

En la tercera sección se propone y desarrolla el sistema de recomendación, donde se abordará una estretegia segun el enunciado.

## Librerias

In [1]:
import pandas as pd


## Carga y Análisis descriptivo de la data

In [6]:
# Carga de datasets a dataframes
commerces_df = pd.read_csv('/content/data/commerces.csv')
product_df = pd.read_csv('/content/data/product.csv')
transactions_df = pd.read_csv('/content/data/transactions.csv')

(commerces_df.head(),product_df.head(),transactions_df.head())

(   id_commerce     district
 0            1  Providencia
 1            2    Penalolen
 2            3    Penalolen
 3            4        Nunoa
 4            5        Nunoa,
    id_product        name     category  price
 0           1  Producto 1        Hogar     54
 1           2  Producto 2  Electrónica     76
 2           3  Producto 3        Hogar     67
 3           4  Producto 4    Alimentos     95
 4           5  Producto 5         Ropa     95,
    id_commerce  id_product  quantity  price
 0           27          17         6    324
 1           64          50         7    693
 2           49           5         4    380
 3           58          22         4    168
 4            9          33         7    343)

In [4]:
# analisis rápido de los data set con info de pandas
commerces_info = commerces_df.info()
product_info = product_df.info()
transactions_info = transactions_df.info()

# aplicamos describe para tener un resumen estadistico
commerces_description = commerces_df.describe()
product_description = product_df.describe()
transactions_description = transactions_df.describe()

# revision de nulos
commerces_nulls = commerces_df.isnull().sum()
product_nulls = product_df.isnull().sum()
transactions_nulls = transactions_df.isnull().sum()


(commerces_info, product_info, transactions_info,
 commerces_description, product_description, transactions_description,
 commerces_nulls, product_nulls, transactions_nulls)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_commerce  100 non-null    int64 
 1   district     100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_product  50 non-null     int64 
 1   name        50 non-null     object
 2   category    50 non-null     object
 3   price       50 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   id_commerce  500000 non-null  int64
 1   id_product   500000 non-null  int64
 2   quantity    

(None,
 None,
 None,
        id_commerce
 count   100.000000
 mean     50.500000
 std      29.011492
 min       1.000000
 25%      25.750000
 50%      50.500000
 75%      75.250000
 max     100.000000,
        id_product      price
 count    50.00000  50.000000
 mean     25.50000  56.600000
 std      14.57738  26.204066
 min       1.00000  10.000000
 25%      13.25000  37.250000
 50%      25.50000  54.500000
 75%      37.75000  80.000000
 max      50.00000  99.000000,
          id_commerce     id_product       quantity          price
 count  500000.000000  500000.000000  500000.000000  500000.000000
 mean       50.540948      25.495818       5.004330     283.329010
 std        28.842359      14.421320       2.585476     206.799043
 min         1.000000       1.000000       1.000000      10.000000
 25%        26.000000      13.000000       3.000000     108.000000
 50%        51.000000      25.000000       5.000000     240.000000
 75%        76.000000      38.000000       7.000000     41

## Top Products por Comuna

In [18]:
#Como uno de los requisitos es ver patrones dentro de la misma comuna, se crearán clusters para cada comuna
commerces_df['district'].unique()


array(['Providencia', 'Penalolen', 'Nunoa', 'Macul', 'La Florida'],
      dtype=object)

In [28]:
# Funciones para obtener los Top Products por comuna con el % mas vendido una para unidades y otra para Monto recaudado
def top_units_por_comuna(commerces_df, district_name, transactions_df, perc=0.2):

    # Filtrar los comercios de la comuna específica
    district_commerces = commerces_df[commerces_df['district'] == district_name]['id_commerce']

    # Filtrar las transacciones de estos comercios
    district_transactions = transactions_df[transactions_df['id_commerce'].isin(district_commerces)]

    # Agrupar por producto y sumar las cantidades vendidas
    product_sales = district_transactions.groupby('id_product')['quantity'].sum().sort_values(ascending=False)

    # Calcular el 20% superior de productos
    top_percent_cutoff = int(len(product_sales) * perc)
    top_products = product_sales.head(top_percent_cutoff)

    return top_products

def top_nmv_por_comuna(commerces_df, district_name, transactions_df, perc=0.2):

    # Filtrar los comercios de la comuna específica
    district_commerces = commerces_df[commerces_df['district'] == district_name]['id_commerce']

    # Filtrar las transacciones de estos comercios
    district_transactions = transactions_df[transactions_df['id_commerce'].isin(district_commerces)]

    # Agrupar por producto y sumar las cantidades vendidas
    product_sales = district_transactions.groupby('id_product')['price'].sum().sort_values(ascending=False)

    # Calcular el 20% superior de productos
    top_percent_cutoff = int(len(product_sales) * perc)
    top_products = product_sales.head(top_percent_cutoff)

    return top_products



In [29]:
# Top Products (unidades) por comuna
top_units_pro = top_units_por_comuna(commerces_df, 'Providencia', transactions_df)
top_units_pen = top_units_por_comuna(commerces_df, 'Penalolen', transactions_df)
top_units_nun = top_units_por_comuna(commerces_df, 'Nunoa', transactions_df)
top_units_mac = top_units_por_comuna(commerces_df, 'Macul', transactions_df)
top_units_flo = top_units_por_comuna(commerces_df, 'La Florida', transactions_df)

(top_units_pro, top_units_pen, top_units_nun, top_units_mac, top_units_flo)

(id_product
 9     10014
 22     9982
 34     9898
 7      9842
 26     9828
 14     9808
 21     9763
 49     9756
 20     9727
 5      9712
 Name: quantity, dtype: int64,
 id_product
 2     13630
 31    13584
 50    13542
 8     13470
 10    13418
 20    13371
 7     13308
 24    13270
 47    13236
 25    13234
 Name: quantity, dtype: int64,
 id_product
 4     11268
 28    10901
 34    10886
 8     10847
 30    10846
 38    10824
 47    10796
 16    10756
 15    10701
 9     10697
 Name: quantity, dtype: int64,
 id_product
 29    8059
 22    7997
 45    7937
 2     7929
 17    7885
 21    7796
 10    7785
 36    7718
 50    7712
 28    7707
 Name: quantity, dtype: int64,
 id_product
 24    10007
 16     9972
 1      9836
 22     9822
 29     9789
 48     9766
 45     9741
 30     9643
 47     9623
 9      9620
 Name: quantity, dtype: int64)

In [30]:
# Top Products (nmv) por comuna
top_nmv_pro = top_nmv_por_comuna(commerces_df, 'Providencia', transactions_df)
top_nmv_pen = top_nmv_por_comuna(commerces_df, 'Penalolen', transactions_df)
top_nmv_nun = top_nmv_por_comuna(commerces_df, 'Nunoa', transactions_df)
top_nmv_mac = top_nmv_por_comuna(commerces_df, 'Macul', transactions_df)
top_nmv_flo = top_nmv_por_comuna(commerces_df, 'La Florida', transactions_df)

(top_nmv_pro, top_nmv_pen, top_nmv_nun, top_nmv_mac, top_nmv_flo)

(id_product
 14    931760
 5     922640
 25    913824
 19    913164
 50    907038
 4     892525
 40    821025
 35    801282
 8     793178
 38    788928
 Name: price, dtype: int64,
 id_product
 50    1340658
 19    1271746
 25    1270464
 4     1252005
 14    1232815
 5     1186645
 40    1174978
 8     1158420
 38    1085448
 7     1064640
 Name: price, dtype: int64,
 id_product
 4     1070460
 50    1042470
 19    1041054
 14    1005195
 25    1003296
 5      986005
 40     933521
 8      932842
 38     909216
 15     877482
 Name: price, dtype: int64,
 id_product
 50    763488
 19    738528
 5     724660
 14    718960
 25    711360
 4     673170
 40    671594
 38    641004
 15    626398
 8     624704
 Name: price, dtype: int64,
 id_product
 19    940702
 50    938619
 4     909245
 25    907392
 5     903165
 14    902310
 40    847013
 8     820612
 35    797630
 38    788592
 Name: price, dtype: int64)

In [85]:
# También podemos incluir un análisis por Categoría y total vendido (nmv) pero lo dejaremos opcional sin mayor desarrollo
def top_products_por_categoria(commerces_df, district_name, transactions_df, product_df, n_top=15):

    district_commerces = commerces_df[commerces_df['district'] == district_name]['id_commerce']
    district_transactions = transactions_df[transactions_df['id_commerce'].isin(district_commerces)]
    district_transactions = district_transactions.merge(product_df[['id_product', 'category']], on='id_product')


    district_transactions['total_value'] = district_transactions['price'] * district_transactions['quantity']
    top_products_by_category = district_transactions.groupby(['category', 'id_product'])['total_value'].sum().reset_index()
    top_products_by_category = top_products_by_category.sort_values(['category', 'total_value'], ascending=[True, False]).groupby('category').head(n_top)

    return top_products_by_category



In [88]:
# top products por comuna y categoría
top_products_cat_pro = top_products_por_categoria(commerces_df, 'Providencia', transactions_df, product_df)
top_products_cat_pen = top_products_por_categoria(commerces_df, 'Penalolen', transactions_df, product_df)
top_products_cat_nun = top_products_por_categoria(commerces_df, 'Nunoa', transactions_df, product_df)
top_products_cat_mac = top_products_por_categoria(commerces_df, 'Macul', transactions_df, product_df)
top_products_cat_flo = top_products_por_categoria(commerces_df, 'La Florida', transactions_df, product_df)

#(top_products_cat_pro,top_products_cat_pen,top_products_cat_nun,top_products_cat_mac,top_products_cat_flo)

## Recomendación por comercio

In [105]:
def recomendacion(id_commerce):
    # Identificar la comuna del comercio
    commerce_district = commerces_df[commerces_df['id_commerce'] == id_commerce]['district'].values[0]
    purchased_products = transactions_df[transactions_df['id_commerce'] == id_commerce]['id_product'].unique()

    if commerce_district == 'Providencia':
        top_units=top_units_pro
        top_nmv=top_nmv_pro

    elif commerce_district == 'Penalolen':
        top_units=top_units_pen
        top_nmv=top_nmv_pen

    elif commerce_district == 'Nunoa':
        top_units=top_units_nun
        top_nmv=top_nmv_nun

    elif commerce_district == 'Macul':
        top_units=top_units_mac
        top_nmv=top_nmv_mac

    elif commerce_district == 'La Florida':
        top_units=top_units_flo
        top_nmv=top_nmv_flo

    rec_nmv = top_nmv[~top_nmv.isin(purchased_products)].index.to_list()
    rec_units = top_units[~top_units.isin(purchased_products)].index.to_list()
    rec_units = list(set(rec_units) - set(rec_nmv)) #Eliminar redundancia de doble recomendaciones en listas



    return commerce_district,rec_nmv,rec_units

### Uso del recomendador

In [107]:
#Ingrese id en la siguiente variable

id_comercio =3 #CAMBIAR ID POR EL QUE NECESITES CONSULTAR

comuna,recomendacion_nmv,recomendacion_unidades=recomendacion(id_comercio)

print('Para el comercio con id '+ str(id_comercio) +' de la comuna '+ comuna +'Se recomiendan los siguientes articulos para aumentar NMV:\n'
+ str(recomendacion_nmv) +'\n'+
'Puedes incluir los siguientes articulos si desea aumentar volumen de unidades vendidas:\n'+ str(recomendacion_unidades))

Para el comercio con id 3 de la comuna PenalolenSe recomiendan los siguientes articulos para aumentar NMV:
[50, 19, 25, 4, 14, 5, 40, 8, 38, 7]
Puedes incluir los siguientes articulos si desea aumentar volumen de unidades vendidas:
[2, 10, 47, 20, 24, 31]
